<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-anya-23-ct/blob/main/1_conv1d_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D Convolution on CPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
# !git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 4), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (7/7), 1.45 KiB | 1024 bytes/s, done.
From https://github.com/ML-HW-SYS/a3-anya-23-ct
   2f1d79c..a11ad81  main       -> origin/main
Updating 2f1d79c..a11ad81
Fast-forward
 4_gemm_gpu.ipynb | 69 ++++++++++++++++++++++++++++++++++-----------------------------------
 src/ops.py       |  2 +-
 2 files changed, 35 insertions(+), 36 deletions(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Verify the following cell prints your github repository.

In [5]:
!ls {PROJECT_ROOT}

1_conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5_conv2d_dw_gpu.ipynb	README.md  tests
2_conv1d_gpu.ipynb  4_gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2. Install TVM

In [ ]:
# !pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 3.4 MB/s eta 0:00:00


In [6]:
!pip install https://github.com/tlc-pack/tlcpack/releases/download/v0.10.0/apache_tvm-0.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 16.9 MB/s eta 0:00:00


## 3. Implement `make_conv1d_cpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_cpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array.
You should return both the TVM schedule and the TVM operator for
1. Input $x$
2. Input $y$
3. Output $out$

The schedule should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells the usage.

In [ ]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler

M = 4096
N = 128
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_cpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "llvm")

dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.06619556 0.28784528 0.34671655 ... 0.4234213  0.07120275 0.00216073]
Output: [0.06619556 0.28784528 0.34671655 ... 0.4234213  0.07120275 0.00216073]
1D conv TVM runtime: 27.193367 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

@main = primfn(A_1: handle, W_1: handle, B_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [4096], []),
             W: Buffer(W_2: Pointer(float32), float32, [128], []),
             B: Buffer(B_2: Pointer(float32), float32, [4223], [])}
  buffer_map = {A_1: A, W_1: W, B_1: B}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [4096], []), W_1: W_3: Buffer(W_2, float32, [128], []), B_1: B_3: Buffer(B_2, float32, [4223], [])} {
  for (n: int32, 0, 4223) {
    B[n] = 0f32
    for (k: int32, 0, 4223) {
      let cse_var_1: int32 = (n - k)
      B[n] = (B[n] + @tir.if_then_else((((4096 <= k) || (cse_var_1 < 0)) || (128 <= cse_var_1)), 0f32, (A[k]*W[cse_var_1]), dtype=float32))
    }
  }
}




In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 15 items                                                                                 

tests/test_1dconv_cpu.py ...............                                                     [100%]

======================================== 15 passed in 8.80s ========================================


After Optimisation

In [16]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler

M = 4096
N = 128
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_cpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "llvm")

dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.02194187 0.480431   1.1462868  ... 0.9548663  1.3068657  0.61126083]
Output: [0.02194187 0.480431   1.1462868  ... 0.9548663  1.3068657  0.61126083]
1D conv TVM runtime: 0.462112 ms


In [17]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

@main = primfn(A_1: handle, W_1: handle, B_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [4096], []),
             W: Buffer(W_2: Pointer(float32), float32, [128], []),
             B: Buffer(B_2: Pointer(float32), float32, [4223], [])}
  buffer_map = {A_1: A, W_1: W, B_1: B}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [4096], []), W_1: W_3: Buffer(W_2, float32, [128], []), B_1: B_3: Buffer(B_2, float32, [4223], [])} {
  allocate(A_pad: Pointer(global float32), float32, [4350]), storage_scope = global {
    for (n: int32, 0, 4350) {
      A_pad_1: Buffer(A_pad, float32, [4350], [])[n] = @tir.if_then_else(((n < 127) || (4223 <= n)), 0f32, A[(n - 127)], dtype=float32)
    }
    for (n.outer: int32, 0, 528) {
      for (n.inner.s: int32, 0, 8) {
        if @tir.likely((((n.outer*8) + n.inner.s) < 4223), dtype=bool) {
          B[((n.outer*8) + n.inner.s)] = 0f

In [18]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 15 items                                                                                 

tests/test_1dconv_cpu.py ...............                                                     [100%]

======================================== 15 passed in 6.24s ========================================


Optim 2

In [7]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler

M = 4096
N = 128
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_cpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "llvm")

dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.19260116 0.4903775  0.51767015 ... 0.43595773 0.00736898 0.07426991]
Output: [0.19260116 0.4903775  0.51767015 ... 0.43595773 0.00736898 0.07426991]
1D conv TVM runtime: 0.271897 ms


In [8]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

@main = primfn(A_1: handle, W_1: handle, B_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [4096], []),
             W: Buffer(W_2: Pointer(float32), float32, [128], []),
             B: Buffer(B_2: Pointer(float32), float32, [4223], [])}
  buffer_map = {A_1: A, W_1: W, B_1: B}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [4096], []), W_1: W_3: Buffer(W_2, float32, [128], []), B_1: B_3: Buffer(B_2, float32, [4223], [])} {
  allocate(A_pad: Pointer(global float32), float32, [4350]), storage_scope = global {
    for (n: int32, 0, 4350) {
      A_pad_1: Buffer(A_pad, float32, [4350], [])[n] = @tir.if_then_else(((n < 127) || (4223 <= n)), 0f32, A[(n - 127)], dtype=float32)
    }
    for (n.outer: int32, 0, 264) {
      for (n.inner.s: int32, 0, 16) {
        if @tir.likely((((n.outer*16) + n.inner.s) < 4223), dtype=bool) {
          B[((n.outer*16) + n.inner.s)] =

In [9]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 15 items                                                                                 

tests/test_1dconv_cpu.py ...............                                                     [100%]

======================================== 15 passed in 7.35s ========================================
